In [1]:
import pandas as pd

In [3]:
df3=pd.read_csv('/Users/liaopeng3/logs/google_analytics_all/train_df_3.csv',dtype={'fullVisitorId': 'str'})

# Show the Values of Columns

# Baseline: Random Forest

In [4]:
df3.columns

Index(['channelGrouping', 'date', 'fullVisitorId', 'sessionId', 'visitId',
       'visitNumber', 'visitStartTime', 'device.browser',
       'device.deviceCategory', 'device.isMobile', 'device.operatingSystem',
       'geoNetwork.city', 'geoNetwork.continent', 'geoNetwork.country',
       'geoNetwork.metro', 'geoNetwork.networkDomain', 'geoNetwork.region',
       'geoNetwork.subContinent', 'totals.hits', 'totals.pageviews',
       'totals.transactionRevenue', 'trafficSource.adContent',
       'trafficSource.adwordsClickInfo.adNetworkType',
       'trafficSource.adwordsClickInfo.gclId',
       'trafficSource.adwordsClickInfo.page',
       'trafficSource.adwordsClickInfo.slot', 'trafficSource.campaign',
       'trafficSource.keyword', 'trafficSource.medium',
       'trafficSource.referralPath', 'trafficSource.source'],
      dtype='object')

In [5]:
train_x = df3.loc[:,['visitNumber',  'totals.hits', 'totals.pageviews',\
              'trafficSource.adwordsClickInfo.page']].fillna(0)
train_y = df3['totals.transactionRevenue'].fillna(0)

In [6]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

regr = RandomForestRegressor(max_depth=2, random_state=0,
                             n_estimators=100)
model=regr.fit(train_x, train_y)

print(regr.feature_importances_)

/Users/liaopeng3/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


[0.51231306 0.         0.48768694 0.        ]


In [7]:

from sklearn import cross_validation, linear_model
import numpy as np


/Users/liaopeng3/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [8]:
def calc_train_error(X_train, y_train, model):
    '''returns in-sample error for already fit model.'''
    predictions = model.predict(X_train)
    mse = mean_squared_error(y_train, predictions)
    rmse = np.sqrt(mse)
    return rmse
    
def calc_validation_error(X_test, y_test, model):
    '''returns out-of-sample error for already fit model.'''
    predictions = model.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    rmse = np.sqrt(mse)
    return mse
    
def calc_metrics(X_train, y_train, X_test, y_test, model):
    '''fits model and returns the RMSE for in-sample error and out-of-sample error'''
    model.fit(X_train, y_train)
    train_error = calc_train_error(X_train, y_train, model)
    validation_error = calc_validation_error(X_test, y_test, model)
    return train_error, validation_error

In [9]:
from sklearn.metrics import mean_squared_error

In [10]:
calc_train_error(train_x, train_y, model)

51081817.65920786

In [12]:
import lightgbm as lgb
d_train = lgb.Dataset(train_x, label=train_y)
# params = {}
# params['learning_rate'] = 0.003
# params['boosting_type'] = 'gbdt'
# params['objective'] = 'binary'
# params['metric'] = 'rmse'
# params['sub_feature'] = 0.5
# params['num_leaves'] = 10
# params['min_data'] = 50
# params['max_depth'] = 10
params = {'num_leaves': 300,
         'min_data_in_leaf': 30, 
         'objective':'regression',
         'max_depth': -1,
         'learning_rate': 0.005,
         "min_child_samples": 20,
         "boosting": "rf",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.8 ,
         "bagging_seed": 11,
         "metric": 'rmse',
         "lambda_l1": 1,
         "verbosity": -1}
clf = lgb.train(params, d_train, 100)
# num_round = 100
# clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=100, early_stopping_rounds = 100)

In [13]:
train_x.dtypes

visitNumber                              int64
totals.hits                              int64
totals.pageviews                       float64
trafficSource.adwordsClickInfo.page    float64
dtype: object

In [14]:
train_x['totals.hits']=train_x['totals.hits'].astype(float)
train_x['totals.pageviews']=train_x['totals.pageviews'].astype(float)
train_x['trafficSource.adwordsClickInfo.page']=train_x['trafficSource.adwordsClickInfo.page'].astype(float)


In [15]:
calc_train_error(train_x, train_y, clf)

50322755.741115525

In [23]:
predictions = model.predict(train_x)

In [26]:
predictions

array([  252180.49264425,   252180.49264425,   252180.49264425, ...,
       17985491.36665884, 21989359.04396007, 40689956.89989892])

In [28]:
from sklearn import linear_model

In [29]:
lm = linear_model.LinearRegression()

In [30]:
lm_model = lm.fit(train_x,train_y)

/Users/liaopeng3/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/base.py:509: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  linalg.lstsq(X, y)


In [31]:
calc_train_error(train_x, train_y, lm_model)

52080039.53443065

In [32]:
predictions = lm_model.predict(train_x)

In [33]:
predictions

array([-1807905.75913015, -1807905.75913015, -1807905.75913015, ...,
       20403768.48553109, 21250786.76467314, 30478262.60880995])

# Baseline 2: Linear Regression

In [34]:
import sklearn

In [35]:
lm = sklearn.linear_model.Lasso()

In [36]:
lm_trained = lm.fit(train_x,train_y)

In [37]:
val_pred_lm = lm_trained.predict(train_x)

In [39]:
calc_train_error(train_x, train_y, lm_trained)

52080039.53443106